# Computing Optimal Labels
Backtesting a Always Buy/Sell Strategies to compute Optimal strategy for a day-ticker combination using MILP

In [ ]:
# !pip install import_ipynb --quiet
# !pip install anvil-uplink --quiet
# !pip install yfinance --quiet
# !pip install pandas_ta --quiet
# !pip install ipynb --quiet
# !pip install rpyc --quiet
# !pip install pulp

In [ ]:
# # pull files from Github
# !git clone https://github.com/gmshroff/algostrats.git

In [ ]:
# %cd algostrats

In [ ]:
# change to True if on colab
colab=False

In [ ]:
import pandas as pd
import numpy as np
import import_ipynb
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate
import pickle
from itertools import product
from tqdm.notebook import tqdm

In [ ]:
from rulestrats import RuleStrat,do_nothing,always_buy,always_sell

In [ ]:
from pulp_jobs import Jobs

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
loadfeed=False
savefeed=False
synthetic=True

Deterministic Strategy - 1 or -1

In [ ]:
class DetStrat(RuleStrat):
    def __init__(self,action=0):
        self.action=action
        self.data_cols=['Close']
        self.model_type='rule-based'
        super().__init__()
    def check_entry_batch(self,dfD):
        if self.action==-1: return always_sell(dfD)
        elif self.action==1: return always_buy(dfD)
        else: return do_nothing(dfD)

In [ ]:
data=pd.read_csv('./capvol100.csv')

In [ ]:
tickers=list(data.iloc[0:50]['ticker'].values)

In [ ]:
if not loadfeed:
    feed=BackFeed(tickers=tickers,nd=3,nw=1,synthetic=synthetic)
    if savefeed: 
        with open('../../temp_data/feed.pickle','wb') as f: pickle.dump(feed,f)
elif loadfeed:
    with open('../../temp_data/feed.pickle','rb') as f: feed=pickle.load(f)

Gather Data for Computing Optimal Strategies for a given Feed

In [ ]:
def assemble_backtests(feed,topk=3,actions=[-1,1],
                       stops=[.01,.005,.02],
                       targets=[.01,.005,.02]):
    posD={}
    print('Assembling backtests')
    for (action,stop,target) in tqdm([c for c in product(actions,stops,targets)]):
        detStrat=DetStrat(action)
        bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=target,stop=stop,txcost=0.001,
                    loc_exit=True,scan=True,topk=3,deploy=False,save_dfs=False)
        bt.run_all(tickers=feed.tickers,model=detStrat,verbose=False)
        for t in bt.results:
            if t not in posD: posD[t]={}
            for d in bt.results[t]:
                if d not in posD[t]: posD[t][d]=[]
                max_r=0
                max_e=0
                for p in bt.results[t][d]['rew']:
                    if p[1]>0 and (p[1]>max_r or p[0]>=max_e): 
                        posD[t][d]+=[(p[0],p[2],p[1],(p[0],action,stop,target,p[2],p[1]))]
                        max_r,max_e=max(max_r,p[1]),max(max_e,p[2])
    return posD

In [ ]:
def compute_optimal(posD):
    optD={}
    for t in posD:
        if t not in optD: optD[t]={}
        for d in posD[t]:
            print(f'Computing optimal strategy for {t} on {d}')
            jobs=Jobs()
            for p in posD[t][d]:
                jobs.add_job(p[0],p[1],p[2],p[3])
            if jobs.k>0:
                jobs.setup()
                jobs.solve()
                x=jobs.get_soln()
                optD[t][d]=[j for i,j in enumerate(jobs.dL) if x[i]>0]
            else: optD[t][d]=[]
    [optD[t][d].sort(key=lambda x: x[0]) for t in optD for d in optD[t]]
    optR={t+' '+d:(lambda x: sum(x) if len(x)>0 else 0)([o[5] for o in optD[t][d]]) for t in optD for d in optD[t]}
    return optD,optR

In [ ]:
loadposD=False

In [ ]:
if loadposD: 
    with open('../../temp_data/posD.pickle','rb') as f: posD=pickle.load(f)
else: 
    posD=assemble_backtests(feed)
    with open('../../temp_data/posD.pickle','wb') as f: pickle.dump(posD,f)

In [ ]:
optD,optR=compute_optimal(posD)

In [ ]:
optD,optR

In [ ]:
optD

## Trading Plots of Optimal Strategy

In [ ]:
import plotly.graph_objects as go

In [ ]:
import plotly.express as px

In [ ]:
def annotate_action(rew,act,df):
    if rew[1]>=0:color='Green'
    else: color='Red'
    if act[0]==1:text='Buy'
    elif act[0]==-1:text='Sell'
    ann=dict(font=dict(color=color,size=15),x=df.index[rew[0]],y=df.iloc[rew[0]]['Close'],
             showarrow=True,text=text)
    return ann

In [ ]:
def annotate_exit(rew,act,anns,df):
    if rew[1]>=0:color='Green'
    else: color='Red'
    X=[a['x'] for a in anns if a is not None]
    if df.index[rew[2]] in X: 
        idx=X.index(df.index[rew[2]])
        anns[idx]['text']='Ex&'+anns[idx]['text']
    else:
        anns+=[dict(font=dict(color=color,size=15),x=df.index[rew[2]],y=df.iloc[rew[2]]['Close'],
                    showarrow=True,text='Exit')]

In [ ]:
def plot_ticker_date(optD,ticker,date):
    global fig
    df=feed.ndata[ticker][date]
    df=df.loc[df['Date']==date]
    fig = go.Figure(data=
        [go.Candlestick(x = df.index,
                        open  = df["Open"],
                        high  = df["High"],
                        low   = df["Low"],
                        close = df["Close"])]
    )
    # reward=np.round(bt.results[ticker][date]["tot"],2)
    reward=(lambda x: sum(x) if len(x)>0 else 0)([o[5] for o in optD[ticker][date]])
    fig.update_layout(
        title=f'{ticker} on {date} return {reward}',
        yaxis_title="Price"
    )
    anns=[]
    for s,a,_,_,e,r in optD[ticker][date]:
        anns+=[annotate_action((s,r,e),(a,s),df)]
    for s,a,_,_,e,r in optD[ticker][date]:
        anns+=[annotate_exit((s,r,e),(a,s),anns,df)]
    
    # for r,a in zip(bt.results[ticker][date]['rew'],bt.results[ticker][date]['acts']):
    #     anns+=[annotate_action(r,a,df)]
    # for r,a in zip(bt.results[ticker][date]['rew'],bt.results[ticker][date]['acts']):
    #     anns+=[annotate_exit(r,a,anns,df)]
    for a in anns: 
        if a is not None: fig.add_annotation(a)
    fig.show()
    return fig

In [ ]:
def combine_plotly_figs_to_html(plotly_figs, html_fname, include_plotlyjs='cdn', 
                                separator=None, auto_open=False):
    with open(html_fname, 'w') as f:
        f.write(plotly_figs[0].to_html(include_plotlyjs=include_plotlyjs))
        for fig in plotly_figs[1:]:
            if separator:
                f.write(separator)
            f.write(fig.to_html(full_html=False, include_plotlyjs=False))

    if auto_open:
        import pathlib, webbrowser
        uri = pathlib.Path(html_fname).absolute().as_uri()
        webbrowser.open(uri)

In [ ]:
figs=[]
for t in optD:
    for d in optD[t]:
        figs+=[plot_ticker_date(optD,t,d)]

In [ ]:
# # upload kaggle.json
# from google.colab import files
# uploaded=files.upload()

In [ ]:
# !mkdir /root/.kaggle
# !mv ./kaggle.json /root/.kaggle/.
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# %mkdir data
# %cd data
# !kaggle datasets download -d gmshroff/marketdatafivemin
# !unzip marketdatafivemin.zip
# %cd ..

In [ ]:
# DATAFILE='augdata_16-Dec-2022_5m.csv'
# if not colab: DATAPATH='~/DataLocal/algo_fin_new/five_min_data/'
# else: DATAPATH='./data/'
# df=pd.read_csv(DATAPATH+DATAFILE)

In [ ]:
# dataFeed=DataFeed(tickers=list(df.ticker.unique()[0:50]),dfgiven=True,df=df)

In [ ]:
# bt=Backtest(dataFeed,tickers=dataFeed.tickers,target=.05,stop=.01,txcost=0.001,
#             loc_exit=True,scan=True,topk=3,deploy=True)

In [ ]:
# bt.run_all(tickers=dataFeed.tickers,model=momStrat)

In [ ]:
# bt.results

In [ ]:
# bt.returns

Experiments